In [1]:
import time
import keyboard
import k3d
import pandas as pd

In [2]:
from sympy import *
def mateq2list(lhs: Matrix, rhs: Matrix):
    L = []
    for i in range(lhs.rows):
        L.append(Eq(lhs[i], rhs[i], evaluate=False))
    return L
## Variables
t = symbols("t")                                                # Tiempo
g = symbols("g")                                                # Constante gravitacional
m = symbols("m")                                                # Masa del cohete
T = symbols("T", cls=Function)                                  # Empuje de propulsor
T_a1, T_a2 = symbols("T_theta, T_phi", cls=Function)            # Orientación de propulsor
q0, q1, q2, q3 = symbols("q0, q1, q2, q3", cls=Function)                                # Orientación del cohete
vx, vy, vz = symbols("v_x, v_y, v_z", cls=Function)             # Velocidad

## Ecuaciones del movimiento en traslación
# Fuerzas Body frame
F_B = Matrix([
    T(t)*cos(T_a1(t)),
    T(t)*sin(T_a1(t))*cos(T_a2(t)),
    T(t)*sin(T_a1(t))*sin(T_a2(t))
])

# Matriz transformación de Body frame a Inertial frame
Q_BI = Matrix([
    [2*q0(t)**2+2*q1(t)**2-1, 2*(q1(t)*q2(t)-q0(t)*q3(t)), 2*(q1(t)*q3(t)+q0(t)*q2(t))],
    [2*(q1(t)*q2(t)+q0(t)*q3(t)), 2*q0(t)**2+q2(t)**2-1, 2*(q2(t)*q3(t)-q0(t)*q1(t))],
    [2*(q1(t)*q3(t)-q0(t)*q2(t)), 2*(q2(t)*q3(t)+q0(t)*q1(t)), 2*q0(t)**2+q3(t)**2-1]
])

# Segunda ley de newton
G = Matrix([0, 0, -g])
V = Matrix([vx(t), vy(t), vz(t)])
eqs_1 = mateq2list(diff(V, t), G + Q_BI*F_B/m)
for eq in eqs_1: display(eq)

Eq(Derivative(v_x(t), t), ((2*q0(t)*q2(t) + 2*q1(t)*q3(t))*T(t)*sin(T_phi(t))*sin(T_theta(t)) + (-2*q0(t)*q3(t) + 2*q1(t)*q2(t))*T(t)*sin(T_theta(t))*cos(T_phi(t)) + (2*q0(t)**2 + 2*q1(t)**2 - 1)*T(t)*cos(T_theta(t)))/m)

Eq(Derivative(v_y(t), t), ((-2*q0(t)*q1(t) + 2*q2(t)*q3(t))*T(t)*sin(T_phi(t))*sin(T_theta(t)) + (2*q0(t)*q3(t) + 2*q1(t)*q2(t))*T(t)*cos(T_theta(t)) + (2*q0(t)**2 + q2(t)**2 - 1)*T(t)*sin(T_theta(t))*cos(T_phi(t)))/m)

Eq(Derivative(v_z(t), t), -g + ((2*q0(t)*q1(t) + 2*q2(t)*q3(t))*T(t)*sin(T_theta(t))*cos(T_phi(t)) + (-2*q0(t)*q2(t) + 2*q1(t)*q3(t))*T(t)*cos(T_theta(t)) + (2*q0(t)**2 + q3(t)**2 - 1)*T(t)*sin(T_phi(t))*sin(T_theta(t)))/m)

In [3]:
## Variables
l = symbols("l")                                                # Largo del cohete
Ix, Iy, Iz = symbols("I_x, I_y, I_z")                           # Momentos de inercia principales
wx, wy, wz = symbols("omega_x, omega_y, omega_z", cls=Function) # Velocidad angular Body frame

## Ecuaciones del movimiento en rotación
# Momentos Body frame
T_R = Matrix([-l/2, 0, 0])  # Posición de propulsor respecto a centro de masa
M_B = T_R.cross(F_B)

# Ecuaciónes de euler para rotación Body frame
I = diag(Ix, Iy, Iz)
w = Matrix([wx(t), wy(t), wz(t)])
eqs_2 = mateq2list(diff(w, t), I**-1*(M_B-w.cross(I*w)))
for eq in eqs_2: display(eq)

# Convertir velocidad angular Body frame a Inertial frame
Q = Matrix([q0(t), q1(t), q2(t), q3(t)])
S = 1/2*Matrix([
    [-q1(t), -q2(t), -q3(t)],
    [q0(t), -q3(t), q2(t)],
    [q3(t), q0(t), -q1(t)],
    [-q2(t), q1(t), q0(t)]
])
eqs_3 = mateq2list(diff(Q, t), S*w)
for eq in eqs_3: display(eq)

Eq(Derivative(omega_x(t), t), (I_y*omega_y(t)*omega_z(t) - I_z*omega_y(t)*omega_z(t))/I_x)

Eq(Derivative(omega_y(t), t), (-I_x*omega_x(t)*omega_z(t) + I_z*omega_x(t)*omega_z(t) + l*T(t)*sin(T_phi(t))*sin(T_theta(t))/2)/I_y)

Eq(Derivative(omega_z(t), t), (I_x*omega_x(t)*omega_y(t) - I_y*omega_x(t)*omega_y(t) - l*T(t)*sin(T_theta(t))*cos(T_phi(t))/2)/I_z)

Eq(Derivative(q0(t), t), -0.5*omega_x(t)*q1(t) - 0.5*omega_y(t)*q2(t) - 0.5*omega_z(t)*q3(t))

Eq(Derivative(q1(t), t), 0.5*omega_x(t)*q0(t) - 0.5*omega_y(t)*q3(t) + 0.5*omega_z(t)*q2(t))

Eq(Derivative(q2(t), t), 0.5*omega_x(t)*q3(t) + 0.5*omega_y(t)*q0(t) - 0.5*omega_z(t)*q1(t))

Eq(Derivative(q3(t), t), -0.5*omega_x(t)*q2(t) + 0.5*omega_y(t)*q1(t) + 0.5*omega_z(t)*q0(t))

In [4]:
## Variables
x, y, z = symbols("x, y, z", cls=Function)  # Posición

## Sistema de ecuaciones completo
R = Matrix([x(t), y(t), z(t)])
eqs_4 = mateq2list(diff(R, t), V)
eqs = eqs_4 + eqs_1 + eqs_2 + eqs_3
eqs_rhs = []
for eq in eqs:
    display(eq)
    eqs_rhs.append(eq.rhs)

Eq(Derivative(x(t), t), v_x(t))

Eq(Derivative(y(t), t), v_y(t))

Eq(Derivative(z(t), t), v_z(t))

Eq(Derivative(v_x(t), t), ((2*q0(t)*q2(t) + 2*q1(t)*q3(t))*T(t)*sin(T_phi(t))*sin(T_theta(t)) + (-2*q0(t)*q3(t) + 2*q1(t)*q2(t))*T(t)*sin(T_theta(t))*cos(T_phi(t)) + (2*q0(t)**2 + 2*q1(t)**2 - 1)*T(t)*cos(T_theta(t)))/m)

Eq(Derivative(v_y(t), t), ((-2*q0(t)*q1(t) + 2*q2(t)*q3(t))*T(t)*sin(T_phi(t))*sin(T_theta(t)) + (2*q0(t)*q3(t) + 2*q1(t)*q2(t))*T(t)*cos(T_theta(t)) + (2*q0(t)**2 + q2(t)**2 - 1)*T(t)*sin(T_theta(t))*cos(T_phi(t)))/m)

...

Eq(Derivative(omega_z(t), t), (I_x*omega_x(t)*omega_y(t) - I_y*omega_x(t)*omega_y(t) - l*T(t)*sin(T_theta(t))*cos(T_phi(t))/2)/I_z)

Eq(Derivative(q0(t), t), -0.5*omega_x(t)*q1(t) - 0.5*omega_y(t)*q2(t) - 0.5*omega_z(t)*q3(t))

Eq(Derivative(q1(t), t), 0.5*omega_x(t)*q0(t) - 0.5*omega_y(t)*q3(t) + 0.5*omega_z(t)*q2(t))

Eq(Derivative(q2(t), t), 0.5*omega_x(t)*q3(t) + 0.5*omega_y(t)*q0(t) - 0.5*omega_z(t)*q1(t))

Eq(Derivative(q3(t), t), -0.5*omega_x(t)*q2(t) + 0.5*omega_y(t)*q1(t) + 0.5*omega_z(t)*q0(t))

In [19]:
Q_IB = Transpose(Q_BI)
abx, aby, abz = symbols("a_x^b, a_y^b, a_z^b", cls=Function)
ax, ay, az = symbols("a_x, a_y, a_z", cls=Function)

ab = Matrix([abx(t), aby(t), abz(t)])
a = Matrix([ax(t), ay(t), az(t)])

a2abeqs = []

for eq in mateq2list(ab, Q_IB * a):
    a2abeqs.append(eq.rhs)
    display(eq)
    
a2ab = lambdify((
    [ax(t), ay(t), az(t)],
    [q0(t), q1(t), q2(t), q3(t)]
    ), a2abeqs)

Eq(a_x^b(t), (-2*q0(t)*q2(t) + 2*q1(t)*q3(t))*a_z(t) + (2*q0(t)*q3(t) + 2*q1(t)*q2(t))*a_y(t) + (2*q0(t)**2 + 2*q1(t)**2 - 1)*a_x(t))

Eq(a_y^b(t), (2*q0(t)*q1(t) + 2*q2(t)*q3(t))*a_z(t) + (-2*q0(t)*q3(t) + 2*q1(t)*q2(t))*a_x(t) + (2*q0(t)**2 + q2(t)**2 - 1)*a_y(t))

Eq(a_z^b(t), (-2*q0(t)*q1(t) + 2*q2(t)*q3(t))*a_y(t) + (2*q0(t)*q2(t) + 2*q1(t)*q3(t))*a_x(t) + (2*q0(t)**2 + q3(t)**2 - 1)*a_z(t))

In [20]:
## Sistema de ecuaciones evaluable numéricamente
sym_ss = [x(t), y(t), z(t),
          vx(t), vy(t), vz(t),
          wx(t), wy(t), wz(t),
          q0(t), q1(t), q2(t), q3(t)]
sym_t = [T(t), T_a1(t), T_a2(t)]
sym_pam = [m, g, l, Ix, Iy, Iz]
sys = lambdify((sym_ss, sym_t, sym_pam), eqs_rhs)

from numpy import *
import quaternion
#https://en.wikipedia.org/wiki/Conversion_between_quaternions_and_Euler_angles
def euler2quat(yaw, pitch, roll):
    cr = cos(roll * 0.5)
    sr = sin(roll * 0.5)
    cp = cos(pitch * 0.5)
    sp = sin(pitch * 0.5)
    cy = cos(yaw * 0.5)
    sy = sin(yaw * 0.5)

    q0 = cr * cp * cy + sr * sp * sy
    q1 = sr * cp * cy - cr * sp * sy
    q2 = cr * sp * cy + sr * cp * sy
    q3 = cr * cp * sy - sr * sp * cy

    return q0, q1, q2, q3

In [50]:
## Integración Runge-Kutta
# https://perso.crans.org/besson/publis/notebooks/Runge-Kutta_methods_for_ODE_integration_in_Python.html
def rungekutta4(f, y0, t, args=()):
    n = len(t)
    y = zeros((n, len(y0)))
    y[0] = y0   
    for i in range(n - 1):
        h = t[i+1] - t[i]
        k1 = f(y[i], t[i], *args)
        k2 = f(y[i] + k1 * h / 2., t[i] + h / 2., *args)
        k3 = f(y[i] + k2 * h / 2., t[i] + h / 2., *args)
        k4 = f(y[i] + k3 * h, t[i] + h, *args)
        y[i+1] = y[i] + (h / 6.) * (k1 + 2*k2 + 2*k3 + k4)
    return y


def sys_wrap(y, t):
    tt = array([0, 0, 0])
    pam = array([100, 9.81, 3.5, 3.6, 103, 103])
    return array(sys(y, tt, pam))

# Vector estado
yaw = 0
pitch = -pi/4*3
roll = 0
qw, qx, qy, qz = euler2quat(yaw, pitch, roll)
ss = array([0, 0, 3.5,
            -275, 0, 275,
            0, -0.04, 0,
            qw, qx, qy, qz])

seconds = 56
frequency = 1/60
steps = int(seconds * 1/frequency)
timee = linspace(0, seconds, steps)
resultado = rungekutta4(sys_wrap, ss, timee)
tdf = pd.DataFrame(timee, columns=["t"])
rdf = pd.DataFrame(resultado, columns = sym_ss)

ab2res = []
for i in range(size(resultado,0)):
	ab2res.append(a2ab(sys_wrap(resultado[i], timee[i])[3:6], resultado[i][9:13]))
adf = pd.DataFrame(ab2res, columns=["ab_x(t)", "ab_y(t)", "ab_z(t)"])

df = tdf.join(rdf).join(adf)
display(df)
df.to_csv("resultado.csv", index=False)

,t,x(t),y(t),z(t),v_x(t),v_y(t),v_z(t),omega_x(t),omega_y(t),omega_z(t),q0(t),q1(t),q2(t),q3(t),ab_x(t),ab_y(t),ab_z(t)
0,0.000000,0.000000,0.0,3.500000,-275.0,0.0,275.000000,0.0,-0.04,0.0,0.382683,0.0,-0.923880,0.0,-6.936718,0.0,6.936718
1,0.016672,-4.584698,0.0,8.083335,-275.0,0.0,274.836451,0.0,-0.04,0.0,0.382375,0.0,-0.924007,0.0,-6.932090,0.0,6.941342
2,0.033343,-9.169396,0.0,12.663942,-275.0,0.0,274.672903,0.0,-0.04,0.0,0.382067,0.0,-0.924135,0.0,-6.927460,0.0,6.945963
3,0.050015,-13.754093,0.0,17.241824,-275.0,0.0,274.509354,0.0,-0.04,0.0,0.381759,0.0,-0.924262,0.0,-6.922826,0.0,6.950581
4,0.066687,-18.338791,0.0,21.816978,-275.0,0.0,274.345805,0.0,-0.04,0.0,0.381451,0.0,-0.924389,0.0,-6.918189,0.0,6.955196
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3355,55.933313,-15381.661209,0.0,39.694299,-275.0,0.0,-273.705805,0.0,-0.04,0.0,-0.663859,0.0,-0.747858,0.0,9.740783,0.0,1.163292
3356,55.949985,-15386.245907,0.0,35.129814,-275.0,0.0,-273.869354,0.0,-0.04,0.0,-0.664108,0.0,-0.747636,0.0,9.741557,0.0,1.156796
3357,55.966657,-15390.830604,0.0,30.562603,-275.0,0.0,-274.032903,0.0,-0.04,0.0,-0.664358,0.0,-0.747415,0.0,9.742326,0.0,1.150299
3358,55.983328,-15395.415302,0.0,25.992665,-275.0,0.0,-274.196451,0.0,-0.04,0.0,-0.664607,0.0,-0.747193,0.0,9.743091,0.0,1.143802
